In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
#import geopandas as gpd
import sklearn

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Exploratory Data Analysis**

Importing the data

In [ ]:
data = pd.read_csv('../input/world-happiness/2015.csv')
data.head()

In [ ]:
data.describe()

In [ ]:
data.columns

In [ ]:
data.info()

There are a total of 159 data entries, with 12 columns each and apparently no missing values

In [ ]:
f,ax=plt.subplots(figsize=(10,10))
sns.heatmap(data.corr(),annot=True)#,linewidths=5,fmt='.1f',ax=ax)

The heat map above shows that hapinness score is positively correlated with mainly the economy, family and health of ecah country. This also shows that hapinnes rank and score are negatively correlated, as previously expected.

In [ ]:
#data_correlation = data.corr()
#data_correlation

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot(a=data['Happiness Score'])#, kde=False)

In [ ]:
#plt.figure(figsize=(10,6))
#sns.distplot(a=data['Economy (GDP per Capita)'])#, kde=False)

In [ ]:
plt.figure(figsize=(17,6))
sns.scatterplot(x=data['Happiness Score'], y=data['Economy (GDP per Capita)'], hue=data['Region'])

In [ ]:
plt.figure(figsize=(17,6))
sns.scatterplot(x=data['Happiness Score'], y=data['Family'], hue=data['Region'])

In [ ]:
plt.figure(figsize=(17,6))
sns.scatterplot(x=data['Happiness Score'], y=data['Health (Life Expectancy)'], hue=data['Region'])

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Average hapiness score for different regions")
sns.barplot(x=data['Happiness Score'], y=data['Region'])

In [ ]:
pd.unique(data.Region)

In [ ]:
data_p = pd.get_dummies(data,columns=['Region'])
data_p.info()
data_p.describe()

In [ ]:
data.columns

In [ ]:
#data_p.drop(['Country','Happiness Rank'], axis=1, inplace=True)

After getting a bit of an understanding on the data set it is not time to start doing some regression, initially starting with a single variable Economy to estimate the Happiness Score of a country using linear regression:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
X = data['Economy (GDP per Capita)'].values.reshape(-1,1)
      # , 'Freedom', 'Trust (Government Corruption)',
      # 'Generosity', 'Dystopia Residual'].values
Y = data ['Happiness Score']

In [ ]:
X.shape, Y.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3, random_state=0)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test.shape, y_test.shape

In [ ]:
linear_reg = LinearRegression(normalize=True).fit(x_train,y_train)
linear_reg

In [ ]:
print("Training score:" ,linear_reg.score(x_train,y_train))

In [ ]:
y_pred = linear_reg.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
print ("Testing score:", r2_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(x_test,y_test, c="black")
plt.plot(x_test,y_pred, c="blue", linewidth=2)
plt.xlabel("Economy (GDP per capita)")
plt.ylabel("Hapiness Score")

Now I will complicate it further and will include the rest of the numerical vaiables in the regression:

In [ ]:
X = data.drop(["Country","Region","Happiness Rank", "Happiness Score"], axis=1)

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3, random_state=0)

In [ ]:
x_train.shape

In [ ]:
linear_reg = LinearRegression(normalize=True).fit(x_train,y_train)
linear_reg

In [ ]:
print("Training score:" ,linear_reg.score(x_train,y_train))

In [ ]:
y_pred = linear_reg.predict(x_test)

In [ ]:
print ("Testing score:", r2_score(y_test, y_pred))

This shows that the **Hapiness Score** can be estimated with a **99.99 % accuracy using the linear regression** model.

I will also perform the regression using alternative models: Decision Tree to compare the accuracy of the estimations:

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso

regressor = DecisionTreeRegressor(random_state=0).fit(x_train,y_train)
regressor
Lasso_reg = Lasso(alpha = 0.1).fit(x_train,y_train)
Lasso_reg

In [ ]:
print("Training score for Decision Tree Regressor:" ,regressor.score(x_train,y_train))
print("Training score for Lasso Regressor:" ,Lasso_reg.score(x_train,y_train))

In [ ]:
y_pred_DT = regressor.predict(x_test)
y_pred_L = Lasso_reg.predict(x_test)

In [ ]:
print ("Testing score for Decision Tree Regressor:", r2_score(y_test, y_pred_DT))
print ("Testing score for Lasso Regressor:", r2_score(y_test, y_pred_L))

On the otherhand, the Decission Tree Regressor is overfitting, as the train data has a very high accuracy (100 %) but that valu decreases substancialy for the test data (73.3 %).